In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv('spam.csv')

In [3]:
df

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [4]:
df.isnull().sum()

Category    0
Message     0
dtype: int64

In [5]:
df.duplicated().sum()

415

In [6]:
df.drop_duplicates(inplace=True,ignore_index=True)

In [7]:
df

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5152,spam,This is the 2nd time we have tried 2 contact u...
5153,ham,Will ü b going to esplanade fr home?
5154,ham,"Pity, * was in mood for that. So...any other s..."
5155,ham,The guy did some bitching but I acted like i'd...


In [8]:
X=df.drop('Category',axis=1)
y=df['Category']

In [9]:
import nltk
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [10]:
lem=WordNetLemmatizer()

In [11]:
corpus=[]
for i in range(len(df)):
    sentence=re.sub('[^a-zA-Z]',' ',df['Message'][i])
    sentence=sentence.lower()
    words=sentence.split()
    words=[lem.lemmatize(word)for word in words if word not in set(stopwords.words('english'))]
    sentence=' '.join(words)
    corpus.append(sentence)

In [12]:
corpus

['go jurong point crazy available bugis n great world la e buffet cine got amore wat',
 'ok lar joking wif u oni',
 'free entry wkly comp win fa cup final tkts st may text fa receive entry question std txt rate c apply',
 'u dun say early hor u c already say',
 'nah think go usf life around though',
 'freemsg hey darling week word back like fun still tb ok xxx std chgs send rcv',
 'even brother like speak treat like aid patent',
 'per request melle melle oru minnaminunginte nurungu vettam set callertune caller press copy friend callertune',
 'winner valued network customer selected receivea prize reward claim call claim code kl valid hour',
 'mobile month u r entitled update latest colour mobile camera free call mobile update co free',
 'gonna home soon want talk stuff anymore tonight k cried enough today',
 'six chance win cash pound txt csh send cost p day day tsandcs apply reply hl info',
 'urgent week free membership prize jackpot txt word claim c www dbuk net lccltd pobox ldnw rw'

# Onehot Representation

In [13]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [14]:
### Vocabulary size
voc_size=5000

In [15]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[1469,
  175,
  2411,
  435,
  1323,
  549,
  2202,
  411,
  422,
  991,
  1238,
  1338,
  414,
  2575,
  157,
  3290],
 [2835, 4887, 4761, 1756, 946, 4208],
 [3825,
  3549,
  2663,
  1421,
  2167,
  4914,
  304,
  299,
  3859,
  1886,
  3430,
  2555,
  4914,
  930,
  3549,
  2236,
  2686,
  4494,
  4982,
  1661,
  3558],
 [946, 2506, 3720, 2385, 520, 946, 1661, 4818, 3720],
 [3783, 129, 1469, 450, 1239, 2730, 3810],
 [4480,
  121,
  1420,
  3323,
  3144,
  945,
  4189,
  3078,
  1742,
  4840,
  2835,
  4948,
  2686,
  3276,
  1466,
  1908],
 [4367, 3384, 4189, 1041, 230, 4189, 2874, 2515],
 [4394,
  3690,
  3067,
  3067,
  3147,
  166,
  4588,
  4753,
  2999,
  1051,
  393,
  3348,
  3633,
  1519,
  1051],
 [3255,
  2957,
  2428,
  1076,
  3543,
  2998,
  2972,
  842,
  4487,
  1383,
  4487,
  2190,
  3417,
  870,
  2957],
 [3492,
  1477,
  946,
  2155,
  3995,
  572,
  1392,
  3010,
  3492,
  2928,
  3825,
  1383,
  3492,
  572,
  4309,
  3825],
 [697, 1216, 2041, 4820, 1774, 4483, 

# Embedding Representation

In [16]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)
print(embedded_docs.shape)

[[   0    0    0 ... 2575  157 3290]
 [   0    0    0 ... 1756  946 4208]
 [3549 2663 1421 ... 4982 1661 3558]
 ...
 [   0    0    0 ...   71 1056 1053]
 [   0    0    0 ... 1746  946 3825]
 [   0    0    0 ... 1148 2001  433]]
(5157, 20)


In [17]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [18]:
X_final=np.array(embedded_docs)
y_final=le.fit_transform(y)

In [19]:
print(X_final.shape)
print(y_final.shape)

(5157, 20)
(5157,)


In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.20, random_state=11)

In [21]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


# Model Training

In [22]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
65/65 [==============================] - 5s 32ms/step - loss: 0.3041 - accuracy: 0.8989 - val_loss: 0.1656 - val_accuracy: 0.9564
Epoch 2/10
65/65 [==============================] - 2s 25ms/step - loss: 0.0859 - accuracy: 0.9804 - val_loss: 0.0808 - val_accuracy: 0.9758
Epoch 3/10
65/65 [==============================] - 2s 24ms/step - loss: 0.0397 - accuracy: 0.9884 - val_loss: 0.0656 - val_accuracy: 0.9806
Epoch 4/10
65/65 [==============================] - 2s 24ms/step - loss: 0.0220 - accuracy: 0.9944 - val_loss: 0.0682 - val_accuracy: 0.9816
Epoch 5/10
65/65 [==============================] - 2s 23ms/step - loss: 0.0180 - accuracy: 0.9952 - val_loss: 0.0643 - val_accuracy: 0.9806
Epoch 6/10
65/65 [==============================] - 2s 24ms/step - loss: 0.0097 - accuracy: 0.9976 - val_loss: 0.0810 - val_accuracy: 0.9797
Epoch 7/10
65/65 [==============================] - 2s 25ms/step - loss: 0.0068 - accuracy: 0.9983 - val_loss: 0.1273 - val_accuracy: 0.9738
Epoch 8/10
65

# Performance Metrics And Accuracy

In [23]:
y_pred=model.predict(X_test)

33/33 [==============================] - 1s 6ms/step


In [24]:
y_pred=(y_pred>0.5)

In [25]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [26]:
confusion_matrix(y_pred,y_test)

array([[894,  14],
       [  7, 117]], dtype=int64)

In [27]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

       False       0.99      0.98      0.99       908
        True       0.89      0.94      0.92       124

    accuracy                           0.98      1032
   macro avg       0.94      0.96      0.95      1032
weighted avg       0.98      0.98      0.98      1032



In [28]:
accuracy_score(y_test,y_pred)

0.9796511627906976